<h1><center>Credit Spread Risk Analysis </center></h1>

# Introduction :Taux de rendement actuariel (TRA)

On considere une obligation  de coupon fixe $c=5$% , un notionnel $N=100$, une maturité $n=5 $ ans et un prix sur le marché $P=94.5$. Le **taux de rendement actuariel TRA** de cette obligation est le taux d'intérêt qui égalise la valeur actuelle des flux de trésorerie futurs (*CAD les coupons et le notionnel*) avec son prix actuel sur le marché $P$.
 
 En d'autres termes :
 
 $$\sum_{i=1}^n\frac{C_f}{(1+TRA)^i}+ \frac{N}{(1+TRA)^n} - P=0$$
 
 ou $$\begin{cases}
      C_f= coupon = N.c & \\
      TRA=\text{taux de rendement actuariel} & 
    \end{cases}$$
 
ce qui veut dire que **TRA** est slolution de l'equation :

$$\sum_{i=1}^n\frac{C_f}{(1+x)^i}+ \frac{N}{(1+x)^n} - P=0$$

On pose la fonction $f$ :
$$f(x)=\sum_{i=1}^n\frac{C_f}{(1+x)^i}+ \frac{N}{(1+x)^n} - P$$


On va chercher alors la racine de la fonction $f$. Pour faire cela nous allons utiliser l'algorithme de Newton Raphson.

------------------------------------------------------------------

**Définiton :**

On utilise l'algorithme de Newton Raphson pour trouver la racine d'une fonction $f$ et est définit itérativement de la manière suivante
 $$\begin{cases}
      x_0=v & \\
      x_{n+1}=x_{n}-\frac{f(x_n)}{f'(x_n)} & 
    \end{cases}$$
    
    
-----------------------------------------------------------------

Avec,
$$f'(x)=\sum_{i=1}^n\frac{-i.C_f}{(1+x)^{i+1}}- \frac{n.N}{(1+x)^{n+1}}$$



L'obligation est au pair si $r=c$. En effet on a, 
 
$$\sum_{i=1}^n\frac{C_f}{(1+r)^i}+ \frac{N}{(1+r)^n} - N=0$$
$$\sum_{i=1}^n\frac{N.c}{(1+r)^i}=N- \frac{N}{(1+r)^n}$$
$$\sum_{i=1}^n\frac{c}{(1+r)^i}=1- \frac{1}{(1+r)^n}$$
$$c.\frac{1}{1+r}.\frac{1-\frac{1}{(1+r)^n}}{1-\frac{1}{1+r}}=1- \frac{1}{(1+r)^n}$$
$$\frac{c}{r}.\big(1- \frac{1}{(1+r)^n}\big)=1- \frac{1}{(1+r)^n}$$
Si et seulement si
$$r=c$$

In [459]:
def f(x, prix, coupon, notionel, maturite):
    CF = sum([coupon / (1 + x) ** i for i in range(1, maturite + 1)])
    notionel_actu = notionel / (1 + x) ** maturite
    return CF + notionel_actu - prix


def df(x, coupon, notionel, maturite):
    dCF = sum([-i * coupon / (1 + x) ** (i + 1) for i in range(1, maturite + 1)])
    dnotionel_actu = -maturite * notionel / (1 + x) ** (maturite + 1)
    return dCF + dnotionel_actu


def newton_raphson(x_0, prix, notionel, coupon, maturite, conf, max_iter):
    x = x_0
    for i in range(max_iter):
        x_new = x - f(x, prix, coupon, notionel, maturite) / df(x, coupon, notionel, maturite)
        if abs(x_new - x) < conf:
            return x_new
        x = x_new
    print("could not find TRA")


def price(x, coupon, notionel, maturite):
    CF = sum([coupon / (1 + x) ** i for i in range(1, maturite + 1)])
    notionel_actu = notionel / (1 + x) ** maturite
    return CF + notionel_actu


c = 0.05
notionel = 100
coupon = c * notionel
maturite = 5
prix = 94.5

tra = newton_raphson(0.1, prix, notionel, coupon, maturite, conf = 10 ** -5, max_iter = 10000)

print(round(price(c, coupon, notionel, maturite),5))
print(round(price(tra, coupon, notionel, maturite),5))

100.0
94.5


# Stripping de la courbe de discount 

Dans cette partie nous allons voir une introduction au pricipe de Stripping de courbes de taux.

Le principe est simple, on aimerait construire la courbe de discount qui mets nos emissions de detes au pair.

Pour cela on considère que le coupon du crédit est de la forme $IBOR+s$ ou $s$ est le spread de crédit.

Etant donné la courbe de taux $IBOR$ jusqu'a la maturité $N=30$ ans, on cherche la coube $D(.)$ qui met toute dete de maturité $n\leq N$ au pair, 

Autrement dit: $\forall n\leq N$, nous allons établir une structure de crédit avec les caractéristiques suivantes :

$$\begin{cases}
      \text{La maturité de la structure de crédit est $n$} & \\
      \text{Le Notionnel est de $100$}& \\
      \text{Le coupon est flotant et consideré comme étant le taux IBOR + $s_n$} & 
    \end{cases}$$
    
  
Les Cash les flows du crédit sont payés aux dates du taux $IBOR$ de la manière suivante:
 
Pour une date $t_i$ avant la maturité $n$ le cash flow est :
 
 $$Cf_i =N.D(t_i).\big(l(t_i)+s_n\big)$$
 
et à la maturité $t_n$ le cash flow est :
  $$Cf_n =N.D(t_n)\big(l(t_n)+s_n\big)+N.D(t_n)$$
  
Donc le prix de la structure de crédit est :

$$\pi=N.\big(\sum_{i=0}^nD(t_i).l(t_i)+s_n\sum_{i=0}^nD(t_i)+D(t_n)\big)$$

Avec,

$$\begin{cases}
      D(t_i)=\frac{1}{(1+x_i)^{t_i}},\text{  $x_i$ le taux d'actualisation à trouver !!} & \\
      l(t_i)=IBOR \text{ à la date $t_i$} & \\
      s_n \text{ : Le spread de funding pour un credit de maturité $n$}
    \end{cases}$$

Pour que notre structure soit au pair à la date de l'émission, $s_n$ doit satisfaire :

$$s_n=\frac{1-D(t_n)-\sum_{i=0}^nD(t_i).l(t_i)}{\sum_{i=0}^nD(t_i)}$$

On itère alors sur les maturités $n\leq N$ de manière croissante et on trouve pour chaque $n$ la solution de l'equation 

$$s_n-\frac{D(t_0)-D(t_n)-\sum_{i=0}^nD(t_i).l(t_i)}{\sum_{i=0}^nD(t_i)}=0$$

**Remarque:**
Dans ce framework l'unconu est la courbe de discount $D(.)$ ! On doit trouver les $(x_i)_{i\leq N}$.

# Analyse de risque par sensibilité

Apres avoir définit la courbe $D(.)$ on aimerait voir quel est l'impact d'un changement du spread de crédit $s$.

On va alors calculer en premier temps la sensibilité au taux de crédit avec notre courbe de discount interne $D(.)$.

On itère sur les maturités de la courbe de taux et on calcule le prix d'un crédit avec notre spread de crédit initial, ensuite on calcule le prix d'un crédit où on choque notre spread de funding avec  10 BP:

$$sensi_{matu}=\frac{Price(\text{Initial Credit Spread})-Price(\text{Initial Credit Spread + 10 BP pour matu})}{10BP}$$



# Ajustement de valorisation

L'un des mandats du Valuation Group est le calcul des differents ajustements de valorisation. 

## Own Credit Market risk adjustment
On suppose maintenant que la vue du marché sur notre spread de crédit est : 

$$\text{Market Credit Spread}=\text{Initial Credit Spread}+\text{Market spread}$$

En génerale les ajustements de valorisation sont additionés à la valorisation initiale (ou principale) pour obtenir la juste valeur du produit comme suit:

$$\text{Juste Valeur}=\text{Valorisation Principale}+\text{Ajustement de valorisation}$$
De maniere génerale, on a
$$f(x)-f(x_0)\approx (x-x_0) f'(x_0)$$
On approche alors la variation du $PnL$ due au spread entre le coupon interne et le coupon du marché pour chaque maturité par 

$$ PnL = Price(\text{Market Credit Spread})-Price(\text{Initial Credit Spread})=\text{Market spread}*sensi$$

## Implicitation du spread de crédit depuis le prix du marché

Etant donné le prix du marché d'un crédit $\pi_{mkt}$ on aimerait trouver $s_{mkt}$ qui satisfait:
$$N.\big(\sum_{i=0}^nD(t_i).l(t_i)+s_{mkt}\sum_{i=0}^nD(t_i)+D(t_n)\big)-\pi_{mkt}=0$$

On pose alors 

$$f(x)=N.\big(\sum_{i=0}^nD(t_i).l(t_i)+x\sum_{i=0}^nD(t_i)+D(t_n)\big)-\pi_{mkt}$$

et 

$$g(x)=\frac{\frac{\pi_{mkt}}{N}-D(t_n)-\sum_{i=0}^nD(t_i).l(t_i)}{\sum_{i=0}^nD(t_i)}$$

On a alors, $f(x)=0$ si et seulement si $g(x)=x$

Or, $\mid g'(x)\mid=0< 1$, donc d'apres le Théorème du point fixe de Picard-Banach il existe $s_{mkt}$ tel que $g(s_{mkt})=s_{mkt}$

Ce qui assure que la suite :
$$\begin{cases}
      x_0=u & \\
      x_{n+1}=g(x_n)
    \end{cases}$$
    
Converge vers $s_{mkt}$.

Dans notre cas $s_{mkt}$ est donné explicitement par:
$$s_{mkt}=\frac{\frac{\pi_{mkt}}{N}-D(t_n)-\sum_{i=0}^nD(t_i).l(t_i)}{\sum_{i=0}^nD(t_i)}$$

**Remarque :** On peut utiliser d'autres algorithmes de recherche de racines pour trouver $s_{mkt}$. 

Par exemple la methode de la secante qui définit la suite $(x_n)_{n\geq 0}$ qui converge vers $s_{mkt}$ de la maniere suivante :

$$\begin{cases}
      x_0=u & \\
      x_1=v\\
      x_{n+1}=x_n-\frac{x_n-x_{n-1}}{f(x_n)-f(x_{n-1})}f(x_n)
    \end{cases}$$
Ou bien Newton Raphson avec 

$$f'(x)= N.\sum_{i=0}^nD(t_i)$$

Ah oui, c'est aussi la sensibilité au spread de crédit ^^

In [31]:
from datetime import datetime as datetime
import pandas as pd
from scipy import optimize
from scipy.interpolate import CubicSpline
from abc import ABC, abstractmethod
from dateutil.relativedelta import relativedelta
import numpy as np
import random
from datetime import timedelta

OneBP = 0.0001


# region Calendar
def CleanMaturity(maturity):
    if '_' in maturity:
        return maturity[1:]
    else:
        return maturity


def CleanMaturityToInt(m):
    return int(CleanMaturity(m)[:-1])


class Kalendar:

    @staticmethod
    def DateAdd(date, maturity):
        maturity = CleanMaturity(maturity)
        if maturity == "ON":
            return date + timedelta(days = 1)
        elif maturity[-1] == 'M':
            nbMonths = CleanMaturityToInt(maturity)
            return Kalendar.AddMonths(date, nbMonths)
        elif maturity[-1] == 'Y':
            nbYears = CleanMaturityToInt(maturity)
            return Kalendar.AddYears(date, nbYears)

    @staticmethod
    def NbDays(DateFrom, DateTo):
        delta = DateTo - DateFrom
        return delta.days

    @staticmethod
    def AddDays(current_date, days_to_add):
        return current_date + timedelta(days = days_to_add)

    @staticmethod
    def AddMonths(current_date, months_to_add):
        return current_date + relativedelta(months = months_to_add)

    @staticmethod
    def AddYears(current_date, years_to_add):
        return current_date + relativedelta(years = years_to_add)


# endregion

# region Splines
class Spline(ABC):
    @abstractmethod
    def Interpolate(self, x):
        pass


class ConstantSpline(Spline):
    def __init__(self, constantvalue):
        self.ConstantValue = constantvalue

    def Interpolate(self, x):
        return self.ConstantValue


class PyCubicSpline(Spline):
    def __init__(self, X, Y):
        self.Spline = CubicSpline(X, Y)

    def Interpolate(self, x):
        return float(self.Spline(x))


# endregion

# region Rates and Rate Curves

class CreditSpreadCurve:
    def __init__(self, refDate, dates, CreditSpreads):
        self.refDate = refDate
        self.NbDays = [Kalendar.NbDays(refDate, date) for date in dates]
        self.CreditSpreadSpline = PyCubicSpline(self.NbDays, CreditSpreads)

    def Interpolate(self, x):
        return self.CreditSpreadSpline.Interpolate(x) * OneBP


class DiscountCurve:
    def __init__(self, refDate, dates, discounts):
        self.refDate = refDate
        self.NbDays = [Kalendar.NbDays(refDate, date) for date in dates]
        if len(dates) == 0:
            self.DiscountSpline = ConstantSpline(0)
        elif len(dates) == 1:
            self.DiscountSpline = ConstantSpline(discounts[0])
        else:
            self.DiscountSpline = PyCubicSpline(self.NbDays, discounts)

    def GetDiscount(self, date):
        if date <= self.refDate:
            return 1
        else:
            time = Kalendar.NbDays(self.refDate, date)
            disc = self.DiscountSpline.Interpolate(time) * OneBP
            return 1 / (1 + disc) ** time


class RateCurve:
    def __init__(self, refDate, dates, rates):
        self.refDate = refDate
        self.NbDays = [Kalendar.NbDays(refDate, date) for date in dates]
        if len(dates) == 0:
            self.RateSplines = ConstantSpline(0)
        elif len(dates) == 1:
            self.RateSplines = ConstantSpline(rates[0])
        else:
            self.RateSplines = PyCubicSpline(self.NbDays, rates)

    def GetRate(self, date):
        time = Kalendar.NbDays(self.refDate, date)
        rate = self.RateSplines.Interpolate(time)
        return rate * OneBP


class Rates:
    @staticmethod
    def SwapFloatingLeg(dates, discountCurve, rates):
        result = sum([discountCurve.GetDiscount(date) * rates.GetRate(date)
                      for date in dates])
        return result

    @staticmethod
    def SwapBPV(dates, discountCurve):
        result = sum([discountCurve.GetDiscount(date) for date in dates])
        return result

    @staticmethod
    def FundingCost(dates, discountCurve, spread):
        fundingCost = spread * Rates.SwapBPV(dates, discountCurve)
        return fundingCost


# endregion

# region Stripping
class StrippingUtils:
    @staticmethod
    def DiscountCurve(refDate, rates, dates):
        if len(rates) < 2:
            if len(rates) == 0:
                curve = DiscountCurve(refDate, [refDate], [0])
            else:
                curve = DiscountCurve(refDate, [dates[0]], [rates[0]])
        else:
            curve = DiscountCurve(refDate, dates, rates)
        return curve


class Stripping:
    @staticmethod
    def StripFunding(refDate, discounts, dates, rateCurve, creditSpread):
        for date in dates:
            index = dates.index(date)
            subDates = dates[:index + 1]
            subDiscounts = discounts[:index + 1]
            spread = creditSpread.Interpolate(Kalendar.NbDays(refDate, subDates[-1]))

            def func(x):
                x = x[0]
                XsubDiscounts = subDiscounts.copy()
                XsubDiscounts[-1] = x
                auxCurve = StrippingUtils.DiscountCurve(refDate, XsubDiscounts, subDates)
                dfStart = auxCurve.GetDiscount(dates[0])
                dfEnd = auxCurve.GetDiscount(subDates[-1])
                leg1 = dfStart - dfEnd
                floating = Rates.SwapFloatingLeg(subDates, auxCurve, rateCurve)
                bpv = Rates.SwapBPV(subDates, auxCurve)
                return spread - (leg1 - floating) / bpv

            sol = optimize.root(func, 0, method = 'hybr')
            rate = sol.x[0]
            discounts[index] = rate

        return StrippingUtils.DiscountCurve(refDate, discounts, dates)


# endregion

# region Own Coupon Scenario
class OwnCouponScenario:
    @staticmethod
    def GetFundingSpread(refDate, BorRates, OwnCoupon, dates):
        CreditSpreads = [OC - Ref for OC, Ref in zip(OwnCoupon, BorRates)]
        creditSpread = CreditSpreadCurve(refDate, dates, CreditSpreads)
        return creditSpread


# endregion

class MathUtils:
    @staticmethod
    def PointFixeDePicardBanach(x0, g, MaxIter):
        x = x0
        for _ in range(MaxIter):
            x_new = g(x)
            if abs(x - x_new) < 10 ** -4:
                return x_new
            x = x_new
        print("didn't find solution")

    @staticmethod
    def Secante(x0, x1, iterMax, f):
        x = [x0, x1]
        iter = 0
        while abs(x[-1] - x[-2]) > 10 ** -4 and iter < iterMax:
            x.append(x[-1] - (x[-1] - x[-2]) / (f(x[-1]) - f(x[-2])) * f(x[-1]))
            iter += 1
        if iter < iterMax:
            return x[-1]
        else:
            print("didn't find solution")

    @staticmethod
    def NewtonRaphson(x_0, f, df, conf, max_iter):
        x = x_0
        for i in range(max_iter):
            x_new = x - f(x) / df(x)
            if abs(x_new - x) < conf:
                return x_new
            x = x_new
        print("didn't find solution")


# region Loan Structure
class LoanUtils:

    @staticmethod
    def GetFundingSpreadFromPrice(refDate, maturities, discountCurve, rateCurve, matu, notional, price,
                                  algo = 'Point Fixe'):
        def func(x):
            tempCredit = LoanStructure(refDate, maturities, discountCurve, rateCurve, x, matu, notional)
            priceTheo = tempCredit.ComputeMtM()
            return priceTheo - price

        def dfunc(x):
            tempCredit = LoanStructure(refDate, Maturities, discountCurve, rateCurve, x, matu, Notional)
            return tempCredit.Notional * Rates.SwapBPV(tempCredit.Schedule, tempCredit.DiscountCurve)

        def g(x):
            tempCredit = LoanStructure(refDate, maturities, discountCurve, rateCurve, x, matu, notional)
            dfEnd = tempCredit.DiscountCurve.GetDiscount(tempCredit.MaturityDate)
            leg = Rates.SwapFloatingLeg(tempCredit.Schedule, tempCredit.DiscountCurve, tempCredit.RateCurve)
            bpv = Rates.SwapBPV(tempCredit.Schedule, tempCredit.DiscountCurve)
            res = (price / notional - dfEnd - leg) / bpv
            return res

        try:
            if algo == 'Point Fixe':
                spread = MathUtils.PointFixeDePicardBanach(0, g, 10000)
            elif algo == 'Newton Raphson':
                spread = MathUtils.NewtonRaphson(0, func, dfunc, 10 ** -4, 10000)
            elif algo == 'Secante':
                spread = MathUtils.Secante(0, 1, 10000, func)
            else:
                spread = None
            if spread is None:
                exit(1)
        except:
            print('fall back')
            sol = optimize.root(func, 0, method = 'hybr')
            spread = sol.x[0]
        return spread


class LoanStructure:
    def __init__(self, refDate, maturities, discountCurve, rateCurve, additiveSpread, matu, notional):
        self.MatuIndex = maturities.index(matu)
        self.refDate = refDate
        self.reDates = [Kalendar.DateAdd(self.refDate, matu) for matu in maturities]
        self.Schedule = self.reDates[:self.MatuIndex + 1]
        self.MaturityDate = self.Schedule[-1]
        self.DiscountCurve = discountCurve
        self.RateCurve = rateCurve
        self.AdditiveSpread = additiveSpread
        self.Notional = notional

    def ComputeMtM(self):
        price = self.Notional * (
                Rates.SwapFloatingLeg(self.Schedule, self.DiscountCurve, self.RateCurve)
                + Rates.FundingCost(self.Schedule, self.DiscountCurve, self.AdditiveSpread)
                + self.DiscountCurve.GetDiscount(self.MaturityDate))
        return price


# endregion

# region Risk Analysis
class RiskAnalysis:
    @staticmethod
    def QuickSensi(refDate, discountCurve, rateCurve, matu, Maturities, Notional, addSprd):
        tempCredit = LoanStructure(refDate, Maturities, discountCurve, rateCurve, addSprd, matu, Notional)
        return tempCredit.Notional * Rates.SwapBPV(tempCredit.Schedule, tempCredit.DiscountCurve) * OneBP

    @staticmethod
    def ComputeSensitivity(refDate, discountCurve, rateCurve, matu, Maturities, Notional, addSprd):
        tempCredit = LoanStructure(refDate, Maturities, discountCurve, rateCurve, addSprd, matu, Notional)
        price = tempCredit.ComputeMtM()
        tempCredit.AdditiveSpread += 10 * OneBP
        newPrice = tempCredit.ComputeMtM()
        delta = (newPrice - price) / 10
        return delta

    @staticmethod
    def ValuationImpactOfFundingChange(refDate, discountCurve, rateCurve, creditSpread, Notional, Maturities):
        Sensitivities = {}

        for matu in Maturities:
            time = Kalendar.NbDays(refDate, Kalendar.DateAdd(refDate, matu))
            additiveSpread = creditSpread.Interpolate(time)
            sensi = RiskAnalysis.QuickSensi(refDate, discountCurve, rateCurve, matu, Maturities, Notional,
                                            additiveSpread)
            Sensitivities[matu] = sensi

        return Sensitivities


# endregion

# region Valuation Adjustment
class ValuationAdjustment:
    @staticmethod
    def ComputeReserve(refDate, OwnCoupon, MarketCoupon, Maturities, Notional, sensitivities, Borrates,
                       algo = 'Point Fixe'):
        result = pd.DataFrame(
            columns = ['Maturity', 'Face Value', 'Market Price', 'Adjusted Price', 'Valuation Adjustment',
                       'Market Funding Spread', 'Theoretical Market Funding Spread'])
        resindx = 0
        MarketRiskSpread = [mc - oc for mc, oc in zip(MarketCoupon, OwnCoupon)]
        MarketFundingSpread = [mc - ref for mc, ref in zip(MarketCoupon, Borrates)]
        
        for matu, sensi in sensitivities.items():
            time = Kalendar.NbDays(refDate, Kalendar.DateAdd(refDate, matu))
            additiveSpread = creditSpread.Interpolate(time)
            index = Maturities.index(matu)
            tempCredit = LoanStructure(refDate, Maturities, InternalDiscountCurve, rateCurve, additiveSpread, matu,
                                       Notional)
            price0 = tempCredit.ComputeMtM()

            spread = MarketRiskSpread[index]
            fundingMarket = MarketFundingSpread[index]
            tempCredit.AdditiveSpread += spread * OneBP
            priceM = tempCredit.ComputeMtM()

            ValAdj = spread * sensi
            AdjustedPrice = price0 + ValAdj

            TheoSpread = LoanUtils.GetFundingSpreadFromPrice(refDate, Maturities, InternalDiscountCurve, rateCurve,
                                                             matu, Notional, priceM, algo) / OneBP

            result.loc[resindx] = [matu, price0, priceM, AdjustedPrice, ValAdj, fundingMarket, TheoSpread]
            resindx += 1

        return result


# endregion

# region Tests

# region Structure Configuration
refDate = datetime(2024, 8, 30)

# IBOR Ref rate
Maturities = ["ON", "_3M", "_1Y", "_3Y", "_5Y", "_10Y", "_15Y", "_20Y", "_25Y", "_30Y"]
Maturitydates = [Kalendar.DateAdd(refDate, maturity) for maturity in Maturities]
BorRates = [350, 382, 437, 498, 530, 584, 642, 591, 543, 496]

# Own Coupon IBor rates are the average of the european banks own coupons)
OwnCoupon = [422, 476, 549, 602, 732, 783, 715, 681, 613, 600]

randFactor = [-1 if random.randint(0, 1) == 0 else 1 for _ in range(len(BorRates))]
discountFactors = [x + factor * np.random.uniform(0, 1, 1) for x, factor in zip(BorRates, randFactor)]

Notional = 100
# endregion

# region Curves Configuration
creditSpread = OwnCouponScenario.GetFundingSpread(refDate, BorRates, OwnCoupon, Maturitydates)

rateCurve = RateCurve(refDate, Maturitydates, BorRates)
# endregion

# region Internal Discount Curve Stripping
InternalDiscountCurve = Stripping.StripFunding(refDate, discountFactors, Maturitydates, rateCurve, creditSpread)

# verification of structure being at par for every maturity
for matu in Maturities:
    if matu != 'ON':
        date = Kalendar.DateAdd(refDate, matu)
        time = Kalendar.NbDays(refDate, date)
        additivespread = creditSpread.Interpolate(time)
        tempCredit = LoanStructure(refDate, Maturities, InternalDiscountCurve, rateCurve, additivespread, matu,
                                   Notional)
        price = tempCredit.ComputeMtM()
        print(f"The price of a {CleanMaturity(matu)} loan with the funding spread {round(additivespread / OneBP, 3)} "
              f"BP is :{round(price, 5)}")
# endregion

# region Risk Analysis

Sensi = RiskAnalysis.ValuationImpactOfFundingChange(refDate, InternalDiscountCurve, rateCurve, creditSpread, Notional,
                                                    Maturities)
print("\n")
print(Sensi)
# endregion

# region Valuation Adjustment
RiskLevel = 50

MarketViewCoupon = [oc + RiskLevel * np.random.normal(0, 1, 1)[0] for oc in OwnCoupon]

ValuationReserve = ValuationAdjustment.ComputeReserve(refDate, OwnCoupon, MarketViewCoupon, Maturities, Notional, Sensi,
                                                      BorRates, 'Point Fixe')

print('\n')
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(ValuationReserve)
# endregion
# endregion

The price of a 3M loan with the funding spread 94.0 BP is :100.0
The price of a 1Y loan with the funding spread 112.0 BP is :100.0
The price of a 3Y loan with the funding spread 104.0 BP is :100.0
The price of a 5Y loan with the funding spread 202.0 BP is :100.0
The price of a 10Y loan with the funding spread 199.0 BP is :100.0
The price of a 15Y loan with the funding spread 73.0 BP is :100.0
The price of a 20Y loan with the funding spread 90.0 BP is :100.0
The price of a 25Y loan with the funding spread 70.0 BP is :100.0
The price of a 30Y loan with the funding spread 104.0 BP is :100.0


{'ON': 0.01, '_3M': 0.019121802214585603, '_1Y': 0.02773625189758782, '_3Y': 0.03588248721594294, '_5Y': 0.04314542696961408, '_10Y': 0.04989297717063434, '_15Y': 0.05677697317004887, '_20Y': 0.06313169196512539, '_25Y': 0.06923833681484791, '_30Y': 0.07477723729083956}


  Maturity  Face Value  Market Price  Adjusted Price  Valuation Adjustment  \
0       ON      104.22       104.695         104.695